In [ ]:
import pickle
import numpy as np
import mplhep
import awkward
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import uproot
import boost_histogram as bh


In [ ]:
physics_process = "qcd"

fi1 = uproot.open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11843.0_baseline/DQM_V0001_R000000001__Global__CMSSW_X_Y_Z__RECO.root")
fi2 = uproot.open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11843.0_mlpf/DQM_V0001_R000000001__Global__CMSSW_X_Y_Z__RECO.root")

In [ ]:
physics_process = "ttbar"
data_mlpf = awkward.Array(pickle.load(open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11834.13_mlpf/out.pkl", "rb")))
data_baseline = awkward.Array(pickle.load(open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11834.13_baseline/out.pkl", "rb")))

fi1 = uproot.open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11834.13_baseline/DQM_V0001_R000000001__Global__CMSSW_X_Y_Z__RECO.root")
fi2 = uproot.open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11834.13_mlpf/DQM_V0001_R000000001__Global__CMSSW_X_Y_Z__RECO.root")

In [ ]:
# physics_process = "singlepi"
# data_mlpf = awkward.Array(pickle.load(open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11688.0_mlpf/out.pkl", "rb")))
# data_baseline = awkward.Array(pickle.load(open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11688.0_baseline/out.pkl", "rb")))

In [ ]:
def cms_label(x0=0.12, x1=0.23, x2=0.67, y=0.90):
    plt.figtext(x0, y,'CMS',fontweight='bold', wrap=True, horizontalalignment='left', fontsize=12)
    plt.figtext(x1, y,'Simulation Preliminary', style='italic', wrap=True, horizontalalignment='left', fontsize=10)
    plt.figtext(x2, y,'Run 3 (14 TeV)',  wrap=True, horizontalalignment='left', fontsize=10)
    
physics_process_str = {
    "ttbar": "$\mathrm{t}\overline{\mathrm{t}}$ events",
    "singlepi": "single $\pi^{\pm}$ events",
    "qcd": "QCD",
}

def sample_label(ax, x=0.03, y=0.98, additional_text="", physics_process=physics_process):
    plt.text(x, y,
        physics_process_str[physics_process]+additional_text,
    va="top", ha="left", size=10, transform=ax.transAxes)


In [ ]:
plt.figure(figsize=(5, 5))
ax = plt.axes()

bins = np.linspace(0, 500, 61)
plt.hist(awkward.flatten(data_baseline["ak4PFJetsCHS"]["pt"]), bins=bins, histtype="step", lw=2, label="PF");
plt.hist(awkward.flatten(data_mlpf["ak4PFJetsCHS"]["pt"]), bins=bins, histtype="step", lw=2, label="MLPF");
plt.yscale("log")
plt.ylim(top=1e5)
cms_label()
sample_label(ax, x=0.02)
plt.xlabel("ak4PFJetsCHS $p_T$ [GeV]")
plt.ylabel("Number of jets")
plt.legend(loc="best")

plt.savefig("ak4jet_pt_{}.pdf".format(physics_process), bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5, 5))

bins = np.linspace(0, 2500, 61)
plt.hist(awkward.flatten(data_baseline["ak4PFJetsCHS"]["energy"]), bins=bins, histtype="step", lw=2, label="PF");
plt.hist(awkward.flatten(data_mlpf["ak4PFJetsCHS"]["energy"]), bins=bins, histtype="step", lw=2, label="MLPF");
plt.yscale("log")
plt.ylim(top=1e5)
cms_label()
sample_label(ax, x=0.02)

plt.xlabel("ak4PFJetsCHS $E$ [GeV]")
plt.ylabel("Number of jets")
plt.legend(loc="best")

plt.savefig("ak4jet_energy_{}.pdf".format(physics_process), bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5, 5))

bins = np.linspace(-6, 6, 101)
plt.hist(awkward.flatten(data_baseline["ak4PFJetsCHS"]["eta"]), bins=bins, histtype="step", lw=2, label="PF");
plt.hist(awkward.flatten(data_mlpf["ak4PFJetsCHS"]["eta"]), bins=bins, histtype="step", lw=2, label="MLPF");
#plt.yscale("log")
cms_label()
sample_label(ax)
plt.ylim(top=2000)
plt.xlabel("ak4PFJetsCHS $\eta$")
plt.ylabel("Number of jets")
plt.legend(loc="best")

plt.savefig("ak4jet_eta_{}.pdf".format(physics_process), bbox_inches="tight")

In [ ]:
color_map = {
    1: "red",
    2: "blue",
    11: "orange",
    22: "cyan",
    13: "purple",
    130: "green",
    211: "black"
}

particle_labels = {
    1: "HFEM",
    2: "HFHAD",
    11: "$e^\pm$",
    22: "$\gamma$",
    13: "$\mu$",
    130: "neutral hadron",
    211: "charged hadron"
    
}

In [ ]:
def draw_event(iev):
    pt_0 = data_mlpf["particleFlow"]["pt"][iev]
    energy_0 = data_mlpf["particleFlow"]["energy"][iev]
    eta_0 = data_mlpf["particleFlow"]["eta"][iev]
    phi_0 = data_mlpf["particleFlow"]["phi"][iev]
    pdgid_0 = np.abs(data_mlpf["particleFlow"]["pdgId"][iev])
    
    pt_1 = data_baseline["particleFlow"]["pt"][iev]
    energy_1 = data_baseline["particleFlow"]["energy"][iev]
    eta_1 = data_baseline["particleFlow"]["eta"][iev]
    phi_1 = data_baseline["particleFlow"]["phi"][iev]
    pdgid_1 = np.abs(data_baseline["particleFlow"]["pdgId"][iev])
    
    plt.figure(figsize=(5, 5))
    ax = plt.axes()
    plt.scatter(eta_0, phi_0, marker=".", s=energy_0, c=[color_map[p] for p in pdgid_0], alpha=0.6)

    pids = [211,130,1,2,22,11,13]
    for p in pids:
        plt.plot([], [], color=color_map[p], lw=0, marker="o", label=particle_labels[p])
    plt.legend(loc=8, frameon=False, ncol=3, fontsize=8)

    cms_label()
    sample_label(ax)
    plt.xlim(-6,6)
    plt.ylim(-5,4)
    plt.xlabel("PFCandidate $\eta$")
    plt.ylabel("PFCandidate $\phi$")
    plt.title("MLPF (trained on PF), CMSSW-ONNX inference", y=1.05)
    plt.savefig("event_mlpf_{}_iev{}.pdf".format(physics_process, iev), bbox_inches="tight")
    plt.savefig("event_mlpf_{}_iev{}.png".format(physics_process, iev), bbox_inches="tight", dpi=300)
    
    plt.figure(figsize=(5, 5))
    ax = plt.axes()
    plt.scatter(eta_1, phi_1, marker=".", s=energy_1, c=[color_map[p] for p in pdgid_1], alpha=0.6)
#     plt.scatter(
#         data_baseline["ak4PFJetsCHS"]["eta"][iev],
#         data_baseline["ak4PFJetsCHS"]["phi"][iev],
#         s=data_baseline["ak4PFJetsCHS"]["energy"][iev], color="gray", alpha=0.3
#     )
    cms_label()
    sample_label(ax)
    plt.xlim(-6,6)
    plt.ylim(-5,4)
    plt.xlabel("PFCandidate $\eta$")
    plt.ylabel("PFCandidate $\phi$")
    plt.title("Standard PF, CMSSW", y=1.05)
    
    pids = [211,130,1,2,22,11,13]
    for p in pids:
        plt.plot([], [], color=color_map[p], lw=0, marker="o", label=particle_labels[p])
    plt.legend(loc=8, frameon=False, ncol=3, fontsize=8)
    
    plt.savefig("event_pf_{}_iev{}.pdf".format(physics_process, iev), bbox_inches="tight")
    plt.savefig("event_pf_{}_iev{}.png".format(physics_process, iev), bbox_inches="tight", dpi=300)

In [ ]:
draw_event(0)

In [ ]:
draw_event(1)

In [ ]:
draw_event(2)

In [ ]:
def plot_dqm(key, title, rebin=None):
    h1 = fi1.get(key).to_boost()
    h2 = fi2.get(key).to_boost()

    fig, (ax1, ax2) = plt.subplots(2, 1)
    plt.sca(ax1)
    if rebin:
        h1 = h1[bh.rebin(rebin)]
        h2 = h2[bh.rebin(rebin)]
        
    mplhep.histplot(h1, yerr=0, label="PF");
    mplhep.histplot(h2, yerr=0, label="MLPF");
    plt.legend(frameon=False)
    plt.ylabel("Number of particles / bin")
    sample_label(ax=ax1, additional_text=", "+title, physics_process=physics_process)

    plt.sca(ax2)
    ratio_hist = h2/h1
    vals_y = ratio_hist.values()
    vals_y[np.isnan(vals_y)] = 0
    plt.plot(ratio_hist.axes[0].centers, vals_y, color="gray", lw=0, marker=".")
    plt.ylim(0,2)
    plt.axhline(1.0, color="black", ls="--")
    plt.ylabel("MLPF / PF")
    
    return ax1, ax2
    
#plt.xscale("log")
#plt.yscale("log")

log10_pt = "$\log_{10}[p_T/\mathrm{GeV}]$"
eta = "$\eta$"

dqm_plots_ptcl = [
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/chargedHadron/chargedHadronLog10Pt",
    "ch.had.", log10_pt, "ch_had_logpt"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/chargedHadron/chargedHadronEta",
    "ch.had.", eta, "ch_had_eta"),
    
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/neutralHadron/neutralHadronLog10Pt",
    "n.had.", log10_pt, "n_had_logpt"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/neutralHadron/neutralHadronPtLow",
    "n.had.", "$p_T$ [GeV]", "n_had_ptlow"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/neutralHadron/neutralHadronPtMid",
    "n.had.", "$p_T$ [GeV]", "n_had_ptmid"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/neutralHadron/neutralHadronEta",
    "n.had.", eta, "n_had_eta"),
    
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/HF_hadron/HF_hadronLog10Pt",
     "HFHAD", log10_pt, "hfhad_logpt"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/HF_hadron/HF_hadronEta",
     "HFHAD", eta, "hfhad_eta"),
    
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/HF_EM_particle/HF_EM_particleLog10Pt",
     "HFEM", log10_pt, "hfem_logpt"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/HF_EM_particle/HF_EM_particleEta",
     "HFEM", eta, "hfem_eta"),
    
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/photon/photonLog10Pt",
    "photon", log10_pt, "photon_logpt"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/photon/photonEta",
    "photon", eta, "photon_eta"),
    
    
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/electron/electronLog10Pt",
    "electron", log10_pt, "electron_logpt"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/electron/electronEta",
    "electron", eta, "electron_eta"),    
    
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/muon/muonLog10Pt",
    "muon", log10_pt, "muon_logpt"),
    ("DQMData/Run 1/ParticleFlow/Run summary/PackedCandidates/muon/muonEta",
    "muon", eta, "muon_eta"),
]

dqm_plots_jetres = [
    ("DQMData/Run 1/ParticleFlow/Run summary/PFJetValidation/CompWithGenJet/mean_delta_et_Over_et_VS_et_",
    "jets", "gen-jet $E_t$", "$\Delta E_t / E_t$"),
]

In [ ]:
for key, title, xlabel, plot_label in dqm_plots_ptcl:
    rh = plot_dqm(key, title)
    plt.xlabel(xlabel)
    cms_label()
    plt.savefig("dqm_{}_{}.pdf".format(plot_label, physics_process), bbox_inches="tight")
    plt.savefig("dqm_{}_{}.png".format(plot_label, physics_process), bbox_inches="tight", dpi=300)

In [ ]:
ax1, ax2 = plot_dqm("DQMData/Run 1/JetMET/Run summary/Jet/Cleanedak4PFJetsCHS/Pt", "ak4PFCHS jets")
ax2.set_xlabel("jet $p_t$ [GeV]")
ax1.set_ylabel("number of jets / bin")
#plt.xscale("log")
#plt.ylim(bottom=1, top=1e4)
ax1.set_yscale("log")
ax1.set_ylim(bottom=1, top=1e5)
#ax2.set_ylim(0,5)

cms_label()
plt.savefig("dqm_jet_pt_{}.pdf".format(physics_process), bbox_inches="tight")
plt.savefig("dqm_jet_pt_{}.png".format(physics_process), bbox_inches="tight", dpi=300)

In [ ]:
ax1, ax2 = plot_dqm("DQMData/Run 1/JetMET/Run summary/Jet/CleanedslimmedJetsPuppi/Pt", "ak4PFPuppi jets")
ax2.set_xlabel("jet $p_t$ [GeV]")
ax1.set_ylabel("number of jets / bin")
#plt.xscale("log")
#plt.ylim(bottom=1, top=1e4)
ax1.set_yscale("log")
ax1.set_ylim(bottom=1, top=1e5)
#ax2.set_ylim(0,5)

cms_label()
plt.savefig("dqm_jet_pt_puppi_{}.pdf".format(physics_process), bbox_inches="tight")
plt.savefig("dqm_jet_pt_puppi_{}.png".format(physics_process), bbox_inches="tight", dpi=300)

In [ ]:
ax1, ax2 = plot_dqm("DQMData/Run 1/JetMET/Run summary/Jet/Cleanedak4PFJetsCHS/Eta", "ak4PFCHS jets")
ax2.set_xlabel("jet $\eta$")
ax1.set_ylabel("number of jets / bin")
#plt.xscale("log")
#plt.ylim(bottom=1, top=1e4)
#ax1.set_yscale("log")
ax1.set_ylim(bottom=0, top=1e3)
#ax2.set_ylim(0,5)

cms_label()
plt.savefig("dqm_jet_eta_{}.pdf".format(physics_process), bbox_inches="tight")
plt.savefig("dqm_jet_eta_{}.png".format(physics_process), bbox_inches="tight", dpi=300)

In [ ]:
ax1, ax2 = plot_dqm("DQMData/Run 1/JetMET/Run summary/Jet/CleanedslimmedJetsPuppi/Eta", "ak4PFPuppi jets")
ax2.set_xlabel("jet $\eta$")
ax1.set_ylabel("number of jets / bin")
#plt.xscale("log")
#plt.ylim(bottom=1, top=1e4)
#ax1.set_yscale("log")
#ax1.set_ylim(bottom=0, top=20)
#ax2.set_ylim(0,5)

cms_label()
plt.savefig("dqm_jet_eta_puppi_{}.pdf".format(physics_process), bbox_inches="tight")
plt.savefig("dqm_jet_eta_puppi_{}.png".format(physics_process), bbox_inches="tight", dpi=300)

In [ ]:
# plot_dqm("DQMData/Run 1/ParticleFlow/Run summary/PFJetValidation/CompWithGenJet/mean_delta_et_Over_et_VS_et_", "AK4 PF jets")
# plt.xlabel("gen-jet $E_t$ [GeV]")
# plt.ylabel("profiled $\mu(\Delta E_t / E_t$)")
# plt.xscale("log")
# plt.ylim(0,3)
# cms_label()
# plt.savefig("dqm_jet_mean_delta_et_Over_et_VS_et.pdf", bbox_inches="tight")

In [ ]:
# plot_dqm("DQMData/Run 1/ParticleFlow/Run summary/PFJetValidation/CompWithGenJet/sigma_delta_et_Over_et_VS_et_", "AK4 PF jets")
# plt.xlabel("gen-jet $E_t$ [GeV]")
# plt.ylabel("profiled $\sigma(\Delta E_t / E_t)$")
# plt.xscale("log")
# plt.ylim(0,10)
# cms_label()
# plt.savefig("dqm_jet_sigma_delta_et_Over_et_VS_et.pdf", bbox_inches="tight")

In [ ]:
ax1, ax2 = plot_dqm("DQMData/Run 1/JetMET/Run summary/METValidation/pfMet/MET", "PFMET", rebin=1)
ax2.set_xlabel("$\sum E_t$ [GeV]")
ax1.set_ylabel("number of events / bin")
#ax1.set_xscale("log")
ax1.set_ylim(bottom=1, top=1000)
ax1.set_yscale("log")
plt.savefig("dqm_met_sumet_{}.pdf".format(physics_process), bbox_inches="tight")
plt.savefig("dqm_met_sumet_{}.png".format(physics_process), bbox_inches="tight", dpi=300)

In [ ]:
# plot_dqm("DQMData/Run 1/ParticleFlow/Run summary/PFMETValidation/CompWithGenMET/profileRMS_delta_et_Over_et_VS_et_", "PFMET")
# plt.xlabel("gen-MET $E_t$ [GeV]")
# plt.ylabel("profiled RMS $\Delta E_t / E_t$")
# plt.xscale("log")
# plt.ylim(0,3)
# cms_label()
# plt.savefig("dqm_met_profileRMS_delta_et_Over_et_VS_et.pdf", bbox_inches="tight")

In [ ]:
# plot_dqm("DQMData/Run 1/ParticleFlow/Run summary/PFMETValidation/CompWithGenMET/profile_delta_et_VS_et_", "PFMET")
# plt.xlabel("gen-MET $E_t$ [GeV]")
# plt.ylabel("profiled $\Delta E_t$ [GeV]")
# plt.xscale("log")
# plt.ylim(0, 80)
# cms_label()
# plt.savefig("dqm_met_delta_et_VS_et.pdf", bbox_inches="tight")

In [ ]:
timing_output = """
Nelem=1600 mean_time=5.92 ms stddev_time=5.03 ms mem_used=1018 MB
Nelem=1920 mean_time=6.57 ms stddev_time=1.01 ms mem_used=1110 MB
Nelem=2240 mean_time=6.92 ms stddev_time=0.81 ms mem_used=1127 MB
Nelem=2560 mean_time=7.37 ms stddev_time=0.66 ms mem_used=1136 MB
Nelem=2880 mean_time=8.17 ms stddev_time=0.56 ms mem_used=1123 MB
Nelem=3200 mean_time=8.88 ms stddev_time=1.09 ms mem_used=1121 MB
Nelem=3520 mean_time=9.51 ms stddev_time=0.65 ms mem_used=1121 MB
Nelem=3840 mean_time=10.48 ms stddev_time=0.93 ms mem_used=1255 MB
Nelem=4160 mean_time=11.05 ms stddev_time=0.87 ms mem_used=1255 MB
Nelem=4480 mean_time=12.07 ms stddev_time=0.81 ms mem_used=1230 MB
Nelem=4800 mean_time=12.92 ms stddev_time=0.89 ms mem_used=1230 MB
Nelem=5120 mean_time=13.44 ms stddev_time=0.75 ms mem_used=1230 MB
Nelem=5440 mean_time=14.07 ms stddev_time=0.78 ms mem_used=1230 MB
Nelem=5760 mean_time=15.00 ms stddev_time=0.84 ms mem_used=1230 MB
Nelem=6080 mean_time=15.74 ms stddev_time=1.05 ms mem_used=1230 MB
Nelem=6400 mean_time=16.32 ms stddev_time=1.30 ms mem_used=1230 MB
Nelem=6720 mean_time=17.24 ms stddev_time=0.99 ms mem_used=1230 MB
Nelem=7040 mean_time=17.74 ms stddev_time=0.85 ms mem_used=1230 MB
Nelem=7360 mean_time=18.59 ms stddev_time=1.04 ms mem_used=1230 MB
Nelem=7680 mean_time=19.33 ms stddev_time=0.93 ms mem_used=1499 MB
Nelem=8000 mean_time=20.00 ms stddev_time=1.06 ms mem_used=1499 MB
Nelem=8320 mean_time=20.55 ms stddev_time=1.13 ms mem_used=1499 MB
Nelem=8640 mean_time=21.10 ms stddev_time=0.90 ms mem_used=1499 MB
Nelem=8960 mean_time=22.88 ms stddev_time=1.24 ms mem_used=1499 MB
Nelem=9280 mean_time=23.44 ms stddev_time=1.14 ms mem_used=1499 MB
Nelem=9600 mean_time=23.93 ms stddev_time=1.04 ms mem_used=1499 MB
Nelem=9920 mean_time=24.75 ms stddev_time=0.91 ms mem_used=1499 MB
Nelem=10240 mean_time=25.47 ms stddev_time=1.33 ms mem_used=1499 MB
Nelem=10560 mean_time=26.29 ms stddev_time=1.33 ms mem_used=1499 MB
Nelem=10880 mean_time=26.72 ms stddev_time=1.18 ms mem_used=1490 MB
Nelem=11200 mean_time=29.50 ms stddev_time=2.60 ms mem_used=1502 MB
Nelem=11520 mean_time=28.50 ms stddev_time=0.91 ms mem_used=1491 MB
Nelem=11840 mean_time=29.11 ms stddev_time=1.14 ms mem_used=1491 MB
Nelem=12160 mean_time=30.01 ms stddev_time=1.15 ms mem_used=1499 MB
Nelem=12480 mean_time=30.55 ms stddev_time=0.94 ms mem_used=1499 MB
Nelem=12800 mean_time=31.31 ms stddev_time=1.08 ms mem_used=1499 MB
Nelem=13120 mean_time=32.61 ms stddev_time=1.19 ms mem_used=1499 MB
Nelem=13440 mean_time=33.37 ms stddev_time=1.01 ms mem_used=1499 MB
Nelem=13760 mean_time=34.13 ms stddev_time=1.18 ms mem_used=1499 MB
Nelem=14080 mean_time=34.73 ms stddev_time=1.40 ms mem_used=1499 MB
Nelem=14400 mean_time=35.79 ms stddev_time=1.70 ms mem_used=2036 MB
Nelem=14720 mean_time=36.68 ms stddev_time=1.37 ms mem_used=2036 MB
Nelem=15040 mean_time=37.17 ms stddev_time=0.97 ms mem_used=2036 MB
Nelem=15360 mean_time=38.73 ms stddev_time=1.19 ms mem_used=2036 MB
Nelem=15680 mean_time=39.80 ms stddev_time=1.04 ms mem_used=2036 MB
Nelem=16000 mean_time=40.87 ms stddev_time=1.46 ms mem_used=1996 MB
Nelem=16320 mean_time=41.89 ms stddev_time=1.01 ms mem_used=1996 MB
Nelem=16640 mean_time=43.36 ms stddev_time=1.08 ms mem_used=1996 MB
Nelem=16960 mean_time=44.87 ms stddev_time=1.35 ms mem_used=1996 MB
Nelem=17280 mean_time=46.04 ms stddev_time=0.96 ms mem_used=1996 MB
Nelem=17600 mean_time=47.96 ms stddev_time=1.47 ms mem_used=1996 MB
Nelem=17920 mean_time=49.01 ms stddev_time=1.35 ms mem_used=1996 MB
Nelem=18240 mean_time=50.04 ms stddev_time=1.34 ms mem_used=1956 MB
Nelem=18560 mean_time=51.34 ms stddev_time=1.49 ms mem_used=1956 MB
Nelem=18880 mean_time=52.16 ms stddev_time=1.20 ms mem_used=1956 MB
Nelem=19200 mean_time=53.19 ms stddev_time=1.20 ms mem_used=1956 MB
Nelem=19520 mean_time=54.03 ms stddev_time=0.96 ms mem_used=1956 MB
Nelem=19840 mean_time=55.68 ms stddev_time=1.05 ms mem_used=1956 MB
Nelem=20160 mean_time=56.88 ms stddev_time=1.12 ms mem_used=1956 MB
Nelem=20480 mean_time=57.49 ms stddev_time=1.50 ms mem_used=1956 MB
Nelem=20800 mean_time=60.40 ms stddev_time=3.51 ms mem_used=1959 MB
Nelem=21120 mean_time=61.30 ms stddev_time=3.90 ms mem_used=1959 MB
Nelem=21440 mean_time=60.74 ms stddev_time=1.05 ms mem_used=1948 MB
Nelem=21760 mean_time=61.66 ms stddev_time=1.29 ms mem_used=1948 MB
Nelem=22080 mean_time=63.35 ms stddev_time=1.11 ms mem_used=1948 MB
Nelem=22400 mean_time=64.70 ms stddev_time=1.16 ms mem_used=1948 MB
Nelem=22720 mean_time=65.63 ms stddev_time=0.95 ms mem_used=1948 MB
Nelem=23040 mean_time=67.09 ms stddev_time=1.02 ms mem_used=1948 MB
Nelem=23360 mean_time=68.40 ms stddev_time=1.15 ms mem_used=1948 MB
Nelem=23680 mean_time=69.76 ms stddev_time=0.88 ms mem_used=1948 MB
Nelem=24000 mean_time=71.55 ms stddev_time=0.94 ms mem_used=1948 MB
Nelem=24320 mean_time=73.04 ms stddev_time=1.46 ms mem_used=1948 MB
Nelem=24640 mean_time=74.53 ms stddev_time=1.28 ms mem_used=1948 MB
Nelem=24960 mean_time=76.03 ms stddev_time=1.07 ms mem_used=1948 MB
Nelem=25280 mean_time=77.59 ms stddev_time=0.88 ms mem_used=1948 MB
"""

In [ ]:
time_x = []
time_y = []
time_y_err = []
gpu_mem_use = []
for line in timing_output.split("\n"):
    if len(line)>0:
        spl = line.split()
        time_x.append(int(spl[0].split("=")[1]))
        time_y.append(float(spl[1].split("=")[1]))
        time_y_err.append(float(spl[3].split("=")[1]))
        gpu_mem_use.append(float(spl[5].split("=")[1]))

In [ ]:
import glob
nelem = []
for fi in glob.glob("../data/TTbar_14TeV_TuneCUETP8M1_cfi/raw/*.pkl"):
    d = pickle.load(open(fi, "rb"))
    for elem in d:
        X = elem["Xelem"][(elem["Xelem"]["typ"]!=2)&(elem["Xelem"]["typ"]!=3)]
        nelem.append(X.shape[0])

In [ ]:
plt.figure(figsize=(5,5))
ax = plt.axes()
plt.hist(nelem, bins=np.linspace(2000,6000,100));
plt.ylabel("Number of events / bin")
plt.xlabel("PFElements per event")
cms_label()
sample_label(ax, physics_process="ttbar")

In [ ]:
plt.figure(figsize=(10, 3))
plt.errorbar(time_x, time_y, yerr=time_y_err, marker=".", label="MLPF")
plt.axvline(np.mean(nelem)-np.std(nelem), color="black", ls="--", lw=1.0, label=r"$t\bar{t}$+PU Run 3")
plt.axvline(np.mean(nelem)+np.std(nelem), color="black", ls="--", lw=1.0)
#plt.xticks(time_x, time_x);
plt.xlim(0,30000)
plt.ylim(0,100)
plt.ylabel("Average runtime per event [ms]")
plt.xlabel("PFElements per event")
plt.legend(frameon=False)
cms_label(x1=0.17, x2=0.8)
plt.savefig("runtime_scaling.pdf", bbox_inches="tight")
plt.savefig("runtime_scaling.png", bbox_inches="tight", dpi=300)

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(time_x, gpu_mem_use, marker=".", label="MLPF")
plt.axvline(np.mean(nelem)-np.std(nelem), color="black", ls="--", lw=1.0, label=r"$t\bar{t}$+PU Run 3")
plt.axvline(np.mean(nelem)+np.std(nelem), color="black", ls="--", lw=1.0)
#plt.xticks(time_x, time_x);
plt.xlim(0,30000)
plt.ylim(0,3000)
plt.ylabel("Maximum GPU memory used [MB]")
plt.xlabel("PFElements per event")
plt.legend(frameon=False, loc=4)
cms_label(x1=0.17, x2=0.8)
plt.savefig("memory_scaling.pdf", bbox_inches="tight")
plt.savefig("memory_scaling.png", bbox_inches="tight", dpi=300)